In [2]:
import sys
sys.path.append('C:\\Users\\vegardso\\PycharmProjects\\master_thesis')
import numpy as np
from active_env.envs.active_network_env import ActiveEnv

# Tutorial showing the functionality of `ActiveEnv`
This notebook gives an introduction to the class `ActiveEnv`. The class follows the structure of a general `gym` environment, and can be used for reinforcement algorithms available in [stable-baselines](https://github.com/hill-a/stable-baselines).

A general gym environment has the following structure:

In [4]:
%timeit ENV = ActiveEnv()

1.34 s ± 131 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
import copy
ENV = ActiveEnv()

In [9]:
%timeit env = copy.deepcopy(ENV)

15.8 ms ± 682 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [74]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding

class FooEnv(gym.Env):
  metadata = {'render.modes': ['human']}

  def __init__(self):
    pass
  def step(self, action):
    pass
  def reset(self):
    pass
  def render(self, mode='human', close=False):
    pass

`ActiveEnv` does not have a `render()` method implemented. This is typically used to visualise a game or some other environment that can be animated

## Brief description of the motivation

The agent is allowed to modify the power consumption at nodes in a distribution system with high peak demand and high production from solar power. The goal of the agent is to reduce the number of current and voltage violations in the grid by increasing/ decreasing the consumption at appropriate times. The increase/decrease in power consumption is intended to be a simplified program for demand response that exploits the residential flexibility in the grid. 

Currently, the implementation is based on a significant simplification:
- <b>Constant flexibility:</b>
The flexibility is assumed to be constant. If the agent increases the consumption in one hour, it has no consequences for the future flexibility in the system. This is not realistic, as increasing the consumption corresponds to turning on electrical equipment, and that equipment can't be turned on again later.


# Basic functionality - parameters
`ActiveEnv` has several parameters that determine the state space, actions, rewards etc. What the specific parameters do is described later in this notebook. The parameters are given as an attribute `env.params`

In [2]:
env = ActiveEnv()
env.params

{'episode_length': 200,
 'reward_terms': ['voltage', 'current', 'imbalance', 'activation'],
 'voltage_weight': 1,
 'current_weight': 0.01,
 'imbalance_weight': 1e-05,
 'activation_weight': 0.1,
 'forecast_horizon': 4,
 'flexibility': 0.1,
 'solar_scale': 0.8,
 'demand_scale': 10,
 'state_space': ['sun', 'demand', 'bus', 'imbalance'],
 'v_upper': 1.05,
 'v_lower': 0.95,
 'i_upper': 90,
 'demand_std': 0.03,
 'solar_std': 0.03,
 'total_imbalance': False,
 'reactive_power': True,
 'imbalance_change': False,
 'one_action': False}

We can change the parameters by calling the `set_parameters` method, where we input a dictionary with the parameters we want to change

In [3]:
env.set_parameters({'episode_length':100,
                  'reward_terms':['voltage']})
env.params

{'episode_length': 100,
 'reward_terms': ['voltage'],
 'voltage_weight': 1,
 'current_weight': 0.01,
 'imbalance_weight': 1e-05,
 'activation_weight': 0.1,
 'forecast_horizon': 4,
 'flexibility': 0.1,
 'solar_scale': 0.8,
 'demand_scale': 10,
 'state_space': ['sun', 'demand', 'bus', 'imbalance'],
 'v_upper': 1.05,
 'v_lower': 0.95,
 'i_upper': 90,
 'demand_std': 0.03,
 'solar_std': 0.03,
 'total_imbalance': False,
 'reactive_power': True,
 'imbalance_change': False,
 'one_action': False}

Some parameters will affect the size of the state space and action space. By changing those, the state space is automatically updated. Let's change the forecast horizon which determines the length of the forecasts (in hours).

First we can see the state space by default consists of 86 variables:

In [4]:
env.observation_space

Box(86,)

Changing the forecast_horizon to 24 hours increases the state space to 126 variables:

In [5]:
env.set_parameters({'forecast_horizon':24})
env.observation_space

Box(126,)

## Power system
<img src= "figures\cigre_network_mv_der.png" style=width:600px;height:600px;>

The electrical power grid used is constructed by the International Council on Large Electric Systems (CIGRE). There is wind power connected to bus bar 7 in the figure above, but this is for simplicity assumed to be solar. In other words, all power production from renewable sources in this net follows solar irradiance in this net. The environment can also be given another `pandapower` network.

The network is an attribute in the `ActiveEnv` class:
 


In [6]:
env = ActiveEnv(seed=3)
net = env.powergrid
net

This pandapower network includes the following parameter tables:
   - bus (15 elements)
   - load (18 elements)
   - sgen (9 elements)
   - switch (8 elements)
   - ext_grid (1 element)
   - line (15 elements)
   - trafo (2 elements)
   - bus_geodata (15 elements)
 and the following results tables:
   - res_bus (15 elements)
   - res_line (15 elements)
   - res_trafo (2 elements)
   - res_ext_grid (1 element)
   - res_load (18 elements)
   - res_sgen (9 elements)

`pandapower` stores information in Pandas DataFrames, and there is a table for each of the listed elements above. For instance, the load table  

In [7]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,Load R1,1,14.99400,3.044662,0.0,0.0,15.300,1.0,True,None
1,Load R3,3,0.27645,0.069285,0.0,0.0,0.285,1.0,True,None
2,Load R4,4,0.43165,0.108182,0.0,0.0,0.445,1.0,True,None
3,Load R5,5,0.72750,0.182329,0.0,0.0,0.750,1.0,True,None
4,Load R6,6,0.54805,0.137354,0.0,0.0,0.565,1.0,True,None
5,Load R8,8,0.58685,0.147078,0.0,0.0,0.605,1.0,True,None
6,Load R10,10,0.47530,0.119121,0.0,0.0,0.490,1.0,True,None
7,Load R11,11,0.32980,0.082656,0.0,0.0,0.340,1.0,True,None
8,Load R12,12,14.99400,3.044662,0.0,0.0,15.300,1.0,True,None
9,Load R14,14,0.20855,0.052268,0.0,0.0,0.215,1.0,True,None


# State space
There are several possible state spaces that can be used to numerically represent the environment. 

<img src= "figures\observation.png" style=width:400px;height:300px;>



## Solar and demand forecast
The state can include the solar and consumption forecast in the system. The solar forecast is generated from satellite-derived solar irradiance in central Norway. The next cell shows how to set the state space to include only the solar forecast. The attribute `env.observation_space` gives information about the state space

In [8]:
env = ActiveEnv()
env.set_parameters({'state_space':['sun']})
env.observation_space

Box(4,)

In this case the shape of the state space is 4 because the forecast by default is 4-hour long. This can be changed by using the `set_parameters` method

In [9]:
env.set_parameters({'forecast_horizon':24})
env.observation_space

Box(24,)

### Global solar irradiance
The solar irradiance is assumed to be equal everywhere in the net, so there is not a unique forecast for each bus bar. However, the solar forecast is scaled up by the nominal values, so the absolute production differs from load to load.

### Forecast uncertainty
No forecasts are perfect, so the actual values should deviate from the forecasted values. The actual values are found by adding a noise term to the forecast. The noise term follows a Gaussian distribution with mean 0 and a standard deviation that is proportional to the forecast. The standard deviation in the forecasts are by default 3%. The uncertainty can be changed:

In [10]:
env.set_parameters({'solar_std':0.1,
                   'demand_std':0.3})

## Bus state
It is possible to include the bus state in the state representation. The bus state includes the voltage magnitude, voltage angle, active effect and reactive effect of each bus in the system. The bus values are found in the `res_bus` DataFrame of the pandapower net 

In [11]:
env.powergrid.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.030000,0.000000,-43.196502,-15.696169
1,0.994133,-6.045087,19.839000,4.637136
2,0.977804,-6.573027,0.000000,0.000000
3,0.951848,-7.414877,0.481700,0.208882
4,0.950094,-7.508922,0.411650,0.108182
5,0.948893,-7.573678,0.697500,0.182329
6,0.947488,-7.649919,0.518050,0.137354
7,0.952199,-7.140044,-1.423500,0.047410
8,0.949187,-7.414180,0.556850,0.147078
9,0.948264,-7.449484,0.543750,0.355578


In [12]:
env = ActiveEnv()
env.set_parameters({'state_space':['bus']})
env.observation_space

Box(60,)

The size of the state space is 4 times the number of buses

## Imbalance state
It is desired that the agent shifts the energy consumption, and does not alter in in absolute magnitude. The agent is therefore penalised for increasing the energy imbalance in the system. The agent must receive some information that tells it if the system is in energy imbalance on not. This is the job of the imbalance state. The imbalance state is a vector where each component is the energy imbalance of the loads in the system. Because there are 18 loads, the size of the imbalance space is 18

In [13]:
env = ActiveEnv()
env.set_parameters({'state_space':['imbalance']})
env.observation_space

Box(18,)

It is also possible to only consider the total imbalance in the net. This is done by setting the parameter <i>total_imbalance</i> equal to True

In [14]:
env.set_parameters({'total_imbalance':True})
env.observation_space

Box(1,)

In [15]:
env._get_obs()

array([0.])

# Action space

<img src= "figures\N64-controller-white.jpg" style=width:300px;height:300px;>


There is one action for each row in load table of the network. The agent can independently change the consumption at each load in an interval of flexibility, for instance by +/- 10 %. The loads are assumed to have a constant power factor. In other words, if the active power increases by 10 %, then the reactive power also increases by 10 %. For the Cigre network, we have the action space $\mathcal{A} = \{a_{i}|\;i=1,...,18\}$, $a_{i} \in [-1,1]$. The action is then scaled by the flexibility (0-100%) and forecasted demand, which determines the change in consumption at each load

The action space is described in `env.action_space`

In [23]:
env.action_space

Box(18,)

The action space (and state space) is a `gym.spaces` object. In `ActiveEnv`, the space is a `Box`, where each dimension is bounded by an upper and lower value. The lower and upper limits are -1 and 1, and each variable can vary continuously in this interval. 

In [75]:
env.action_space.high

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.], dtype=float32)

In [76]:
env.action_space.low

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.], dtype=float32)

The `sample()` method can be used to sample a random action uniformly from the action space. 

In [31]:
env.action_space.sample()

array([ 0.58953696,  0.660232  ,  0.69953644,  0.4281357 , -0.57135296,
        0.5132497 ,  0.68536156,  0.5646868 , -0.10707809,  0.65083647,
        0.27660847, -0.6086434 , -0.9253837 ,  0.37014306,  0.7565592 ,
       -0.45291057,  0.3864012 , -0.5078616 ], dtype=float32)

### How the action affects the net
`ActiveEnv` takes in the action vector $\in \mathbb{R^{18}}$, manipulates the `net.load` DataFrame, and solves the powerflow equation. In this example, each load increases its load as much as possible. The flexibility is 10 %.

First, we create an environment instance, and look at the demand forecast. The uncertainty in the forecast is set to 0 to remove stochasticity. 

In [82]:
env = ActiveEnv(seed=3)
env.set_parameters({'demand_std':0})
forecast = env.demand_forecasts[:,0]
print('forecasted demand: ', forecast)

forecasted demand:  [0.45447803]


Next, we create the action vector of ones, which means that all loads increase their consumption as much as possible

In [83]:
a = np.ones(env.action_space.shape)
a
#env.step(a)



array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

The action interacts with the environment using the `step()` method

In [84]:
_,_,_,_ =env.step(a)

The values for consumption and production in the net are now updated, and one hour has passed in the episode. The consumption as a percentage of nominal values is equal at each load

In [85]:
consumption = env.powergrid.res_load['p_mw']/env.powergrid.load['sn_mva']
consumption

0     0.499926
1     0.499926
2     0.499926
3     0.499926
4     0.499926
5     0.499926
6     0.499926
7     0.499926
8     0.499926
9     0.499926
10    0.499926
11    0.499926
12    0.499926
13    0.499926
14    0.499926
15    0.499926
16    0.499926
17    0.499926
dtype: float64

The consumption is 10 % higher than the forecast at every load because the flexibility is 10 %

In [86]:
consumption/forecast

0     1.1
1     1.1
2     1.1
3     1.1
4     1.1
5     1.1
6     1.1
7     1.1
8     1.1
9     1.1
10    1.1
11    1.1
12    1.1
13    1.1
14    1.1
15    1.1
16    1.1
17    1.1
dtype: float64

### Smaller action space
18 free variables in an action space is quite large. It is possible to set a global action that modifies all the loads in the interval of flexibility. This is done shown in the cell below. Note that the change is a percentage change in consumption and the loads have different nominal consumption levels. Therefore, the absolute power change varies from load to load. 

In [87]:
env.set_parameters({'one_action':True})

The size of the action space is now 1

In [92]:
env.action_space

Box(1,)

# Rewards
Rewards are fundamental to all reinforcement learning algorithms. In this case, it is simpler to talk about costs, rather than rewards. The reward is defined as the negative of the cost, so maximising the reward is the same as minimising the cost. Different reward terms can be specified in the `ActiveEnv` class.
<img src= "figures\reward.png" style=width:400px;height:400px;>


### Voltage cost
The goal of the agent is to reduce the number of current and voltage violations in the net. Therefore, one must define what a violation is. There is a voltage violation in the net if the voltage magnitude somewhere steps out of the safety bound. The safety bounds can be specified and are by default 0.95 - 1.05 pu. Currently, the voltage cost for one bus is proportional to the voltage violation, as shown in the figure below. The total voltage cost is found by summing over all bus bars.  

<img src= "figures\voltage_cost.png" style=width:500px;height:370px;>

There is a weight that scales total voltage cost, whose default value is 1. The next cell shows how to only include the voltage in the reward calculation and change the voltage weight


In [124]:
env = ActiveEnv(seed=3)
env.set_parameters({'reward_terms':['voltage'],
                   'voltage_weight':3})

The lower and upper limit can be changed by using the `set_parameters()` method

In [95]:
env.set_parameters({'v_lower':0.9,
                   'v_upper':1.2})

### Current cost
Similarly, there is a current violation in a line if the current exceeds a capacity threshold, which by default is set to 90%. The cost is proportional to the violation in the line, and the total cost is found by summing over all lines. It might be more appropriate to have a non-linear relation, so that one large violation is punished more than several small violations.

<img src= "figures\current_cost.png" style=width:500px;height:370px;>

There is also a weight for the current cost, which is 0.01 by default.


In [125]:
env = ActiveEnv(seed=3)
env.set_parameters({'reward_terms':['current'],
                   'current_weight':0.05})

### Imbalance cost
The imbalance cost penalises a large energy imbalance. The agent should ideally increase power consumption during peak solar production and less during peak demand. The imbalance cost is there to incentives the agent to shift the energy consumption. There are two variations of the imbalance cost:

- <b>Absolute energy imbalance:</b> The agent is penalised proportionally to the energy imbalance over the last 24 hours.


- <b>Increasing energy imbalance:</b> The agent is penalised proportionally to the increase in absolute energy imbalance. For instance, consider a scenario where the imbalance is +20 MWh, and that it increases to +25 MWh and after the agent has performed its action. The imbalance cost is then $(25 - 20)*\texttt{imbalance weight}$. This cost becomes negative when the imbalance decreases, and is the only time the agent can get a positive reward


The default version is the <u>absolute energy imbalance</u> . The next cell shows how to switch to the imbalance change, and how to alter the imbalance weight, which by default is $10e-5$


In [131]:
env = ActiveEnv(seed=3)
env.set_parameters({'reward_terms':['imbalance'],
                   'imbalance_change':True,
                   'imbalance_weight':10e-3})

### Activation cost
Flexibility is unfortunately not a free resource. Currently, the activation cost is proportional to the absolute power change resulting for the agent's action, with a constant price. A more realistic environment should include the cost in the state space, so the agent can learn that it is expensive to change the consumptions when the price is high. The default activation weight is 0.1

In [133]:
env = ActiveEnv(seed=3)
env.set_parameters({'reward_terms':['activation'],
                   'activation_weight':10e-3})

### Total cost
<b>The total cost is found by summing the voltage, current, imbalance and activation cost<b/>
    
<img src= "figures\total_cost.png" style=width:800px;height:130px;>


# Training a reinforcement agent with stable-baselines
Stable-baselines simplifies reinforcement learning in Python, just like sklearn simplifies supervised learning. You can easily test different reinforcement algorithms and policies. Most of the RL algorithms support multiprocessing. 


In [73]:
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines.common.policies import MlpPolicy
from gym_power.envs.active_network_env import ActiveEnv

env = ActiveEnv()
env = DummyVecEnv([lambda: env])
model = PPO2(MlpPolicy, env, verbose=2)
model.learn(total_timesteps=1000)


-------------------------------------
| approxkl           | 0.008776205  |
| clipfrac           | 0.13085938   |
| explained_variance | 0.00713      |
| fps                | 46           |
| n_updates          | 1            |
| policy_entropy     | 25.545404    |
| policy_loss        | -0.036377445 |
| serial_timesteps   | 128          |
| time_elapsed       | 0            |
| total_timesteps    | 128          |
| value_loss         | 3.8982704    |
-------------------------------------
-------------------------------------
| approxkl           | 0.0030996634 |
| clipfrac           | 0.033203125  |
| explained_variance | 0.0295       |
| fps                | 63           |
| n_updates          | 2            |
| policy_entropy     | 25.555183    |
| policy_loss        | -0.017375557 |
| serial_timesteps   | 256          |
| time_elapsed       | 2.75         |
| total_timesteps    | 256          |
| value_loss         | 28.427862    |
-------------------------------------
------------

Save the trained model

In [58]:
model.save('models\ppo_agent')

Let the trained agent play:

In [84]:
obs = env.reset()
for i in range(10):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
